In [ ]:
# FIRST MODEL: apriori

In [ ]:
!pip install apyori
import pandas as pd
import pickle
from tqdm import tqdm
from apyori import apriori
tqdm.pandas()

In [ ]:
# csv containing product names
names = pd.read_csv('https://raw.githubusercontent.com/medinaltbx/datathones/main/capgemini_mercadona/data/raw/tabla_producto.csv',sep=';')
dc_names = {}
for row in names.itertuples():
  dc_names[str(row.id_producto)] = row.nombre_producto

In [ ]:
# initial load
df1 = pd.read_csv('https://raw.githubusercontent.com/medinaltbx/datathones/main/capgemini_mercadona/data/raw/tabla_compras_1.csv',sep=';')
df2 = pd.read_csv('https://raw.githubusercontent.com/medinaltbx/datathones/main/capgemini_mercadona/data/raw/tabla_compras_2.csv',sep=';')
df3 = pd.read_csv('https://raw.githubusercontent.com/medinaltbx/datathones/main/capgemini_mercadona/data/raw/tabla_compras_3.csv',sep=';')
df4 = pd.read_csv('https://raw.githubusercontent.com/medinaltbx/datathones/main/capgemini_mercadona/data/raw/tabla_compras_4.csv',sep=';')
df5 = pd.read_csv('https://raw.githubusercontent.com/medinaltbx/datathones/main/capgemini_mercadona/data/raw/tabla_compras_5.csv',sep=';')

df = pd.concat([df1,df2,df3,df4,df5])

In [ ]:
# dictionary containing list of products by ticket id
def make_dc_records(df):
  dc = {}
  def fill(x):
    id = x['id_orden'].iloc[0]
    dc[id] = x['id_producto'].unique().tolist()
  df.groupby(by=['id_orden']).progress_apply(fill)
  
  return dc

dc = make_dc_records(df)

100%|██████████| 131209/131209 [00:20<00:00, 6271.06it/s]


In [ ]:
def make_apriori(dc):
  lista = [dc[x] for x in dc]
  results = list(apriori(lista,min_support=0.005,min_confidence=0.2))
  return results

results = make_apriori(dc)

In [ ]:
# transformation of model output
def make_apriori_results(results):
  df = pd.DataFrame(columns=('Items','Antecedent','Consequent','Support','Confidence','Lift'))

  Support =[]
  Confidence = []
  Lift = []
  Items = []
  Antecedent = []
  Consequent=[]

  for RelationRecord in results:
      for ordered_stat in RelationRecord.ordered_statistics:
          Support.append(RelationRecord.support)
          Items.append(RelationRecord.items)
          Antecedent.append(ordered_stat.items_base)
          Consequent.append(ordered_stat.items_add)
          Confidence.append(ordered_stat.confidence)
          Lift.append(ordered_stat.lift)

  df['Items'] = list(map(set, Items))                                   
  df['Antecedent'] = list(map(set, Antecedent))
  df['Consequent'] = list(map(set, Consequent))
  df['Support'] = Support
  df['Confidence'] = Confidence
  df['Lift']= Lift
  return df


df_apriori = make_apriori_results(results)
df_apriori.head(10)

,Items,Antecedent,Consequent,Support,Confidence,Lift
0,"{24852, 4605}",{4605},{24852},0.008163,0.284689,1.994754
1,"{4920, 24852}",{4920},{24852},0.008856,0.286277,2.005883
2,"{5450, 24852}",{5450},{24852},0.006593,0.278762,1.953228
3,"{13176, 5876}",{5876},{13176},0.008132,0.304422,2.580293
4,"{47209, 5876}",{5876},{47209},0.005952,0.222825,4.008856
5,"{13176, 8174}",{8174},{13176},0.005526,0.366162,3.103598
6,"{13176, 8277}",{8277},{13176},0.005236,0.305062,2.585717
7,"{8424, 24852}",{8424},{24852},0.007050,0.315484,2.210530
8,"{13176, 8518}",{8518},{13176},0.006135,0.210843,1.787116
9,"{9076, 24852}",{9076},{24852},0.005457,0.308222,2.159645


PARTE ALGORITMO RECOMENDADOR

In [ ]:
# SECOND MODEL: word2vec

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from time import time
from gensim.models import Word2Vec 

In [ ]:
def prepare_data(dc):
  for k in dc:
    dc[k] = [str(x) for x in dc[k]]
  records = list(dc.values())
  random.shuffle(records)
  stop = int(len(records)*0.8)
  train, test = records[:stop], records[stop:]
  return records, train, test

records, train, test = prepare_data(dc)

In [ ]:
# train word2vec model
def create_model(records):
  t = time()
  model = Word2Vec(window = 10,sg = 1,hs = 0,negative = 10,alpha=0.03,min_alpha=0.0007,seed = 14)
  model.build_vocab(records, progress_per=200)
  model.train(records,total_examples = model.corpus_count,epochs=10,report_delay=1)
  print('TIME: ', time()-t)
  model.init_sims(replace=True)
  return model
model = create_model(records)

TIME:  179.63672375679016


In [ ]:
def get_similar_prods(prod_prueba,model,n_elements=5):

  sim_words = model.wv.most_similar(prod_prueba, topn=20)
  res = pd.DataFrame({'Producto':[],'Confianza':[]})

  for x in sim_words:
    res = pd.concat([res, pd.DataFrame({'Producto':[dc_names[x[0]]],'Confianza':[x[1]]})])

  res.reset_index(drop=True, inplace=True)
  if len(res)>n_elements:
    res = res[:n_elements]
  print(f'PRODUCTOS QUE ESTAN RELACIONADOS CON PRODUCTO CON ID = {prod_prueba} ({dc_names[prod_prueba]})')
  return res

prod_prueba = '1' # IDENTIFICADOR DEL PRODUCTO
n_elements = 5    # NUMERO DE PRODUCTOS RECOMENDADOS

keys = list(model.wv.vocab.keys())
if prod_prueba in list(model.wv.vocab.keys()):
  print(get_similar_prods(prod_prueba = prod_prueba, n_elements = n_elements, model=model))
else:
  print('THIS KEY DOES NOT HAVE RECOMMENDED PRODUCTS. THESE ARE SOME AVAILABLE KEYS: ', keys[:20])

PRODUCTOS QUE ESTAN RELACIONADOS CON PRODUCTO CON ID = 1 (GALLETAS DEL EMPAREDADO DEL CHOCOLATE)
                                     Producto  Confianza
0                    CHEEZ-IT CHEDDAR CRACKER   0.918878
1            NUTRI GRAIN BARRAS MULTI PAQUETE   0.897148
2               SABOR COPA DE FIDEOS DE POLLO   0.886783
3                       PEPPERONI BAGEL BITES   0.876092
4  CINE MANTEQUILLA DE PALOMITAS DE MICROONDA   0.874172
